# Index

- B*Tree 인덱스는 나뭇잎으로 무성한 나무를 뒤집어 놓은 듯한 모습
- Root에서 Leaf 블럭까지의 거리를 깊이 (Height) 라고 부르며, 인덱스의 반복 탐색시 성능에 영향을 미치는 요소
- Root / Branch 블럭은 하위 노드들의 데이터 값 범위를 나타내는 Key 값과, 키 값에 해당하는 블록 주소 정보를 가지고 있음

- Leaf 블럭은 인덱스 키 값을 가지고, 그 키값에 해당하는 테이블 레코드를 찾아갈 때 필요한 주소 정보(row id)를 가짐 
    - 같은 키 값일때 row id순으로 정렬
    - 인덱스 키(key) 값 순으로 정렬되어 있어 Range Scan이 가능하고 정방향/역방향 (ASC/DESC) 스캔이 가능한 양방향 연결 리스트 구조!
    
    

- **Range 범위가 넓은 경우엔, Index를 타는 것보다 Full Scan이 빠른 경우도 있음!**



- Random IO : 블락 1개에서 데이터 1개 접근
- Sequential IO : 블락 1개에서 데이터 여러개 접근

## 1.1 목표 ( 인덱스 => 정렬)
1. 수직적 Search 효율화
2. 수평적 탐색 선택도 최대화
3. 테이블 Random Access ( Random IO ) 최소화


- 수직적 Search : 수평적 탐색을 위한 시작 지점을 찾는 과정 
- 수평적 탐색 : Leaf 블럭에 저장된 레코드끼리 연결된 순서에 따른 스캔


* 비용 : 테이블 Random IO > 수직적 Search > 수평적 탐색